# Testdata creation
## Table of Contents
1. Create artificial data functions
2. Create testdata   
    > 2.1 Training data    
    > 2.2 Validation data    
    > 2.3 Prediction data    
    > 2.4 Extra files    
4. Look into one file

## 1. Create artificial data functions

In [1]:
import h5py
import numpy as np
import random
import math
import os

In [2]:
# seed to generate the same test data as was generated here
def set_seed(seed):
    os.environ["PL_GLOBAL_SEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)

The function ``create_fake_fasta`` is used to generate a fake fasta file. The genome sequence is randomly generated. The input parameters are ``filename``, the basename for the fasta file, and ``chromosome_map``, a dictionary containing the chromosome information. An example for ``chromosome_map`` would be:    
```python
{"Chrom1": {"length": 51321, "Ns": [5, 34]},
 "Chrom2": {"length": 102642, "Ns": [21640, 43402]}}
```
The dictionary defines the name of each chromosome, i.e., Chrom1 and Chrom2. For each chromosome the total chromosome length and the locations of N bases are defined. The N bases are either defined by a list of start and end position of the N bases/stretch or by setting it to ``None``, which would mean that this chromosome doesn't contain N bases.

In [3]:
def create_fake_fasta(filename, chromosome_map):
    bases = np.array(["A", "T", "G", "C"])
    chromosomes = []
    i = 1
    for chrom, map_ in chromosome_map.items():
        chromosomes.append(f">{chrom} fake chromosome {i}")
        c = np.random.choice(bases, size=map_["length"], replace=True)
        if map_["Ns"] is not None:
            c[map_["Ns"][0]:map_["Ns"][1]] = "N"
        c = np.split(c, np.arange(80, len(c), 80))  # lines of a fasta file are always 80 characters long
        c = ["".join(list(arr)) for arr in c]
        chromosomes.append("\n".join(c))
        i += 1
    
    with open(f"{filename}.fa", "w") as f:
        f.write("\n".join(chromosomes))


The function ``create_testdata`` is used to generate fake h5 files for testing. The test files use the same format as the h5 files created from fasta files with Helixer. The input parameters ``filename`` and ``chromosome_map`` are the same as for ``create_fake_fasta``. The ``species`` parameter can be any fake species name. The ``subsequence_length`` can be any length, the default is 21384 bp, Helixer's standard ``subsequence_length``. An example for ``dataset_info`` would be:    
```python
{"atacseq": 2, "h3k4me3": 4}
```
The dictionary defines the prefix of each dataset and the amount of bam files per dataset.

In [4]:
def create_testdata(filename, dataset_info, species, chromosome_map, subsequence_length=21384):
    h5df = h5py.File(f"{filename}.h5", "w")
    h5df.create_group("data")
    h5df.create_group("evaluation")
    
    # generate X/DNA sequence
    total_chunks = 0
    X = []
    seqids = []
    start_ends = []
    dsets = [[] for _ in range(len(dataset_info))]  # empty list for each dataset
    for chrom, map_ in chromosome_map.items():
        chunks = math.ceil(map_["length"]/subsequence_length)
        total_chunks += chunks*2
        
        # Create DNA sequence (data/X)
        # -----------------------------
        # flat array at first
        x = np.eye(4)[np.random.choice(4, (chunks * subsequence_length))]
        # generate fake padding (is 'added' when the chromosome is not exactly divisible by the sequence length)
        x[map_["length"]:(chunks * subsequence_length)] = [0., 0., 0., 0.]
        # add Ns
        if map_["Ns"] is not None:
            x[map_["Ns"][0]:map_["Ns"][1]] = [0.25, 0.25, 0.25, 0.25]
        
        # if the chromosome is exactly divisible, flipping is not needed
        need_to_flip = map_["length"] % subsequence_length != 0

        x = np.concatenate([x, np.flip(x)])  # reverse strand: np.flip(x)
        x = np.reshape(x, (chunks*2, subsequence_length, 4))

        # flip according to Helixer convention (padded bases are always at the end)
        if need_to_flip:
            padding_end = np.where(np.sum(x[chunks], axis=1) == 0)[0][-1] + 1
            x[chunks] = np.concatenate([x[chunks][padding_end:], x[chunks][:padding_end]])
        X.append(x)
        
        # Create seqids
        # --------------
        seqids.append(np.full(shape=(chunks*2,), fill_value=chrom, dtype="S50"))
        
        # Create start_ends
        # ------------------
        se = np.array([[i, i + subsequence_length] for i in range(0, map_["length"], subsequence_length)])
        se[-1][1] = map_["length"]
        start_ends.append(np.concatenate([se, np.flip(se)]))  # reverse strand: np.flip(se)
        
        # Create datasets (exaluation/<dataset>_coverage)
        # -------------------------------------------------
        for i, dset in enumerate(dataset_info):
            # dataset_info[dset] amount of bam files
            fake_data = np.random.randint(low=0, high=450, size=(chunks * subsequence_length, dataset_info[dset]))
            fake_data[map_["length"]:(chunks * subsequence_length)] = -1
            fake_data = np.concatenate([fake_data, np.flip(fake_data)])
            fake_data = np.reshape(fake_data, (chunks*2, subsequence_length, dataset_info[dset]))

            if need_to_flip:
                fake_data[chunks] = np.concatenate([fake_data[chunks][padding_end:], fake_data[chunks][:padding_end]])
            dsets[i].append(fake_data)
    
    # Create h5 file
    # ----------------
    # data
    h5df.create_dataset("data/X", shape=(total_chunks, subsequence_length, 4), maxshape=(None, subsequence_length, None),
                        dtype="<f2", compression="gzip", compression_opts=9, data=np.concatenate(X))
    h5df.create_dataset("data/seqids", shape=(total_chunks,), maxshape=(None,), dtype="S50", compression="gzip",
                        compression_opts=9, data=np.concatenate(seqids))
    h5df.create_dataset("data/species", shape=(total_chunks,), maxshape=(None,), dtype="S50", compression="gzip",
                        compression_opts=9, data=np.full(shape=(total_chunks,), fill_value=species, dtype="S50"))
    h5df.create_dataset("data/start_ends", shape=(total_chunks, 2), maxshape=(None, 2), dtype="int", compression="gzip",
                        compression_opts=9, data=np.concatenate(start_ends))
    
    # evaluation
    for i, dset in enumerate(dataset_info):
        h5df.create_dataset(f"evaluation/{dset}_coverage", shape=(total_chunks, subsequence_length, dataset_info[dset]),
                            maxshape=(None, subsequence_length, None), dtype="int", compression="lzf",
                            data=np.concatenate(dsets[i]))
        bam_files = [f"fake_{i}.bam" for j in range(dataset_info[dset])]
        h5df.create_dataset(f"evaluation/{dset}_meta/bam_files", shape=(dataset_info[dset], ), maxshape=(None, ),
                            dtype="S512", data=np.array(bam_files, dtype="S512"))
        
    h5df.close()


In [5]:
def add_blacklist(file, sequence_ids):
    sequence_ids = np.array([id_.encode() for id_ in sequence_ids])
    h5df = h5py.File(file, "a")
    chunks = h5df["data/X"].shape[0]
    # True: chunk will be retained, False: chunk is masked/blacklisted
    mask = ~np.in1d(h5df["data/seqids"], sequence_ids)
    h5df.create_dataset("data/blacklist", shape=(chunks,), maxshape=(None,), dtype=np.int32,
                        compression="gzip", compression_opts=9, data=mask)
    h5df.close()

## 2. Create testdata

In [6]:
set_seed(998)

### 2.1 Training data

In [7]:
create_testdata("train_data_1",
                {"atacseq": 2, "h3k4me3": 4}, "Species_artificialis_1",
                {"Chrom1": {"length": 88045, "Ns": [666, 890]},
                 "Chrom2": {"length": 22447, "Ns": None},
                 "Chrom3": {"length": 34566, "Ns": None}})

In [8]:
create_testdata("train_data_2",
                {"atacseq": 1, "h3k4me3": 2, "moaseq": 2}, "Species_artificialis_2",
                {"Chrom1": {"length": 78412, "Ns": [42755, 64160]},
                 "Chrom2": {"length": 32122, "Ns": None},
                 "Unplaced1": {"length": 20489, "Ns": None},
                 "Unplaced2": {"length": 18504, "Ns": [91, 107]}})

In [9]:
create_testdata("train_data_3",
                {"moaseq": 1}, "Species_artificialis_3",
                {"Chrom1": {"length": 42768, "Ns": None},
                 "Chrom2": {"length": 21384, "Ns": None}})

In [10]:
create_testdata("train_data_4",
                {"atacseq": 3, "h3k4me3": 1}, "Species_artificialis_4",
                {"Chrom1": {"length": 58633, "Ns": None},
                 "Chrom2": {"length": 19870, "Ns": None},
                 "Unplaced1": {"length": 1223, "Ns": None},
                 "Unplaced2": {"length": 4587, "Ns": None},
                 "Unplaced3": {"length": 12356, "Ns": None}})

In [11]:
add_blacklist("train_data_4.h5", ["Unplaced1", "Unplaced2", "Unplaced3"])

In [32]:
# special: unplaced scaffold in between chromosomes
create_testdata("train_data_5",
                {"atacseq": 2}, "Species_artificialis_5",
                {"Chrom1": {"length": 104748, "Ns": [20962, 43006]},
                 "Unplaced0_chrom1": {"length": 7655, "Ns": [1222, 1299]},
                 "Chrom2": {"length": 40976, "Ns": [21, 34]},
                 "Unplaced1": {"length": 6982, "Ns": None},
                 "Plastid1": {"length": 9557, "Ns": None}})

In [33]:
add_blacklist("train_data_5.h5", ["Unplaced0_chrom1", "Unplaced1", "Plastid1"])

### 2.2 Validation data

In [16]:
create_testdata("val_data_1",
                {"atacseq": 1, "h3k4me3": 2}, "Species_artificialis_6",
                {"Chrom1": {"length": 64152, "Ns": None},
                 "Chrom2": {"length": 21384, "Ns": None}})

In [17]:
create_testdata("val_data_2",
                {"atacseq": 1, "h3k4me3": 1, "moaseq": 1}, "Species_artificialis_7",
                {"Chrom1": {"length": 20588, "Ns": None},
                 "Chrom2": {"length": 17978, "Ns": None}})

In [18]:
create_testdata("val_data_3",
                {"atacseq": 2}, "Species_artificialis_8",
                {"Chrom1": {"length": 85245, "Ns": [42600, 64231]}})

### 2.3 Prediction data

In [19]:
create_fake_fasta("pred_data",
                  {"Chrom1": {"length": 51321, "Ns": [5, 34]},
                   "Chrom2": {"length": 102642, "Ns": [21640, 43402]}})

The file ``pred_data.fa`` was converted to two h5 files using Helixer's ``fasta2h5.py``. The two h5 files are ``pred_data_1.h5`` with a subsequence length of 21384 bp, and ``pred_data_2.h5`` with a subsequence length of 42768 bp. The chosen species was ``Species_artificialis_9``.

### 2.4 Extra files
Additional test data created was the ``mini_model.ckpt``, a very bare bones model checkpoint that was created using all the training and validation data with this command:
```bash
Predmoter.py -m train -i data -o out --prefix mini -e 5 -b 10 --hidden-size 16 --filter-size 16 \
--seed 5 --num-devices 1 --num-workers 4
# all other parameters were the default parameters
```
The last/fifth model checkpoint was chosen as the ``mini_model.ckpt``. This checkpoint was created with Predmoter version 0.3.2 commit 3ddadf9.   
The one prediction file ``pred_data_1_predictions.h5`` was also created with the same Predmoter version and the command:
```bash
Predmoter.py -m predict -f pred_data_1.h5 --prefix mini -b 10 --model mini_model.ckpt
```

## 4. Look into one file

A closer look into the file ``train_data_4.h5`` is shown here:

In [20]:
h5df = h5py.File("train_data_4.h5", "r")

In [21]:
h5df["data/X"].shape

(14, 21384, 4)

In [22]:
h5df["data/start_ends"][:]

array([[    0, 21384],
       [21384, 42768],
       [42768, 58633],
       [58633, 42768],
       [42768, 21384],
       [21384,     0],
       [    0, 19870],
       [19870,     0],
       [    0,  1223],
       [ 1223,     0],
       [    0,  4587],
       [ 4587,     0],
       [    0, 12356],
       [12356,     0]])

In [23]:
h5df["data/seqids"][:]

array([b'Chrom1', b'Chrom1', b'Chrom1', b'Chrom1', b'Chrom1', b'Chrom1',
       b'Chrom2', b'Chrom2', b'Unplaced1', b'Unplaced1', b'Unplaced2',
       b'Unplaced2', b'Unplaced3', b'Unplaced3'], dtype='|S50')

In [24]:
h5df["data/blacklist"][:]

array([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0])

In [25]:
h5df["data/X"][0][:10]  # the first 50 bp

array([[0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.]], dtype=float16)

In [26]:
h5df["evaluation"].keys()

<KeysViewHDF5 ['atacseq_coverage', 'atacseq_meta', 'h3k4me3_coverage', 'h3k4me3_meta']>

In [27]:
h5df["evaluation/atacseq_coverage"][0][:10]  # the artificial ATAC-seq coverage of the first 50 bp

array([[436, 281, 425],
       [106, 409, 296],
       [256, 189, 223],
       [227, 316,  50],
       [449, 157,  69],
       [106, 335, 364],
       [393, 393, 120],
       [293, 413, 335],
       [ 91, 406,  46],
       [371, 122,  39]])

In [28]:
h5df.close()

A loointo the blacklist array of ``train_data_5.h5``:

In [29]:
h5df = h5py.File("train_data_5.h5", "r")

In [30]:
h5df["data/blacklist"][:]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0])

In [31]:
h5df.close()